In [2]:
import pandas as pd

# Read data from files 
train = pd.read_csv( "./data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )
test = pd.read_csv( "./data/testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "./data/unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

start_data = int(len(train) / 100)

train = train[:(start_data-1)]
test = test[:(start_data-1)]
unlabeled_train = unlabeled_train[:(start_data-1)]

# Verify the number of reviews that were read (100,000 in total)
print("Read %d labeled train reviews, %d labeled test reviews, and %d unlabeled reviews\n" % \
(train["review"].size, test["review"].size, unlabeled_train["review"].size ))

Read 249 labeled train reviews, 249 labeled test reviews, and 249 unlabeled reviews



In [3]:
# Import various modules for string cleaning
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [4]:
# Download the punkt tokenizer for sentence splitting
import nltk.data
#nltk.download()

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [5]:
sentences = []  # Initialize an empty list of sentences

print("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)


Parsing sentences from training set


D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 170 of the file D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


Parsing sentences from unlabeled set


In [6]:
# Check how many sentences we have in total - should be around 850,000+
print(len(sentences))
print(sentences[0])
print(sentences[1])


5104
['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']
['maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent']


In [9]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2018-04-13 16:46:29,503 : INFO : collecting all words and their counts
2018-04-13 16:46:29,505 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-13 16:46:29,580 : INFO : collected 12183 word types from a corpus of 115617 raw words and 5104 sentences
2018-04-13 16:46:29,595 : INFO : Loading a fresh vocabulary
2018-04-13 16:46:29,612 : INFO : min_count=40 retains 320 unique words (2% of original 12183, drops 11863)
2018-04-13 16:46:29,621 : INFO : min_count=40 leaves 75134 word corpus (64% of original 115617, drops 40483)
2018-04-13 16:46:29,657 : INFO : deleting the raw counts dictionary of 12183 items
2018-04-13 16:46:29,678 : INFO : sample=0.001 downsamples 71 most-common words
2018-04-13 16:46:29,701 : INFO : downsampling leaves estimated 40964 word corpus (54.5% of prior 75134)


Training model...


2018-04-13 16:46:29,828 : INFO : estimated required memory for 320 words and 300 dimensions: 928000 bytes
2018-04-13 16:46:29,831 : INFO : resetting layer weights
2018-04-13 16:46:29,900 : INFO : training model with 4 workers on 320 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2018-04-13 16:46:30,082 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-13 16:46:30,085 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-13 16:46:30,088 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-04-13 16:46:30,091 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-04-13 16:46:30,095 : INFO : EPOCH - 1 : training on 115617 raw words (40928 effective words) took 0.1s, 338764 effective words/s
2018-04-13 16:46:30,300 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-04-13 16:46:30,311 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-04-

In [10]:
model.doesnt_match("man woman child kitchen".split())

D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if __name__ == '__main__':
2018-04-13 16:46:37,333 : WARNING : vectors for words {'child', 'woman', 'kitchen'} are not present in the model, ignoring these words


'man'

In [11]:
model.doesnt_match("france england germany berlin".split())

D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if __name__ == '__main__':
2018-04-13 16:46:41,302 : WARNING : vectors for words {'germany', 'france', 'england', 'berlin'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [12]:
model.doesnt_match("paris berlin london austria".split())

D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  if __name__ == '__main__':
2018-04-13 16:46:53,516 : WARNING : vectors for words {'paris', 'austria', 'london', 'berlin'} are not present in the model, ignoring these words


ValueError: cannot select a word from an empty list

In [13]:
model.most_similar("man")

D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


[('who', 0.999769926071167),
 ('has', 0.9997625946998596),
 ('an', 0.9997272491455078),
 ('a', 0.999721884727478),
 ('as', 0.9997215270996094),
 ('with', 0.9996534585952759),
 ('and', 0.9996178150177002),
 ('by', 0.9996095299720764),
 ('which', 0.9995276927947998),
 ('father', 0.9995024800300598)]

In [14]:
model.most_similar("queen")

D:\WinPython-32bit-3.4.4.6Qt5\python-3.4.4\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  if __name__ == '__main__':


KeyError: "word 'queen' not in vocabulary"

In [ ]:
model.most_similar("awful")